In [1]:
from PIL import Image
import sys
import os
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.misc import imsave, imread
import numpy as np

In [2]:
#Padding
def pad_one_im(direc_in, name, direc_out, pad_n):
    img_file = direc_in + name
    img = Image.open(img_file)
    old_size = img.size
    new_size = (old_size[0] + pad_n, old_size[1]+pad_n)
    new_im = Image.new("RGB", new_size)
    new_im.paste(img, ((new_size[0] - old_size[0])/2, (new_size[1]-old_size[1])/2))
    out_file = name[0:(len(name)-4)] + "padded.png"
    new_im.save(direc_out + out_file)
    
def pad_directory(direc_in, direc_out, pad_n):
    imgs = os.listdir(direc_in)
    for i in imgs:
        if '.DS_Store' not in i:
            pad_one_im(direc_in, i, direc_out, pad_n)

In [3]:
#Black & White
def makeItGray(direc_in, name, direc_out):
    img_file = direc_in + name
    img = Image.open(img_file)
    gray = img.convert('LA')
    out_file = name[0:(len(name)-4)] + "_gray.png"
    gray.save(direc_out + out_file)
    
def makeDirGray(direc_in, direc_out):
    imgs = os.listdir(direc_in)
    for i in imgs:
        if '.DS_Store' not in i:
            makeItGray(direc_in, i, direc_out)

In [8]:
#Color Balancing

##taken fromhttps://vzaguskin.github.io/histmatching1/

def imhistmatch(picture, tint, out):
    imsrc = imread(picture)
    imtint = imread(tint)

    nbr_bins=255
    if len(imsrc.shape) < 3:
        imsrc = imsrc[:,:,np.newaxis]
        imtint = imtint[:,:,np.newaxis]

    imres = imsrc.copy()
    for d in range(imsrc.shape[2]):
        imhist,bins = np.histogram(imsrc[:,:,d].flatten(),nbr_bins,normed=True)
        tinthist,bins = np.histogram(imtint[:,:,d].flatten(),nbr_bins,normed=True)

        cdfsrc = imhist.cumsum() #cumulative distribution function
        cdfsrc = (255 * cdfsrc / cdfsrc[-1]).astype(np.uint8) #normalize

        cdftint = tinthist.cumsum() #cumulative distribution function
        cdftint = (255 * cdftint / cdftint[-1]).astype(np.uint8) #normalize


        im2 = np.interp(imsrc[:,:,d].flatten(),bins[:-1],cdfsrc)



        im3 = np.interp(im2,cdftint, bins[:-1])

        imres[:,:,d] = im3.reshape((imsrc.shape[0],imsrc.shape[1] ))

    try:
        imsave(out, imres)
    except:
        imsave(out, imres.reshape((imsrc.shape[0],imsrc.shape[1] )))
        
def imhistmatch_direc(direc_in, direc_out, template):
    imgs = os.listdir(direc_in)
    for i in imgs:
        if '.DS_Store' not in i:
            imhistmatch(direc_in + i, template, direc_out + i)

In [5]:
direc_in_list = ['../data/raw/train/' + d + '/' for d in os.listdir('../data/raw/train')
                if '.DS_Store' not in d]

In [7]:
direc_gray_out_list = ['../data/processed/gray/'+ d + '/'  for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]
#for d in range(0, len(direc_in_list)):
#    makeDirGray(direc_in_list, direc_gray_out_list)

In [8]:
padded_out_list = ['../data/processed/padded/'+ d + '/'  for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]

#pad_directory(direc_in_list, padded_out_list, 50)

In [9]:
gray_in_list = direc_gray_out_list

padded_gray_out_list = ['../data/processed/gray_padded/'+ d + '/'  for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]

#pad_directory(gray_in_list, padded_gray_out_list, 50)

In [10]:
for d in range(0, len(direc_in_list)):
    makeDirGray(direc_in_list[d], direc_gray_out_list[d])
    pad_directory(direc_in_list[d], padded_out_list[d], 50)
    pad_directory(gray_in_list[d], padded_gray_out_list[d], 50)

In [ ]:
#test images
makeDirGray('../data/raw/test_stg1/', '../data/processed/test/gray/')
pad_directory('../data/raw/test_stg1/', '../data/processed/test/padded/', 50)

In [12]:
pad_directory('../data/processed/test/gray/', '../data/processed/test/gray_and_padded/', 50)

In [6]:
direc_balanced_out_list = ['../data/processed/histmatch/'+ d + '/'  for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]
balanced_in_list = direc_balanced_out_list
padded_balanced_out_list = ['../data/processed/histmatch_padded/'+ d + '/'  for d in os.listdir('../data/raw/train')
                        if '.DS_Store' not in d]

In [9]:
for d in range(0, len(direc_in_list)):
    imhistmatch_direc(direc_in_list[d], direc_balanced_out_list[d], "../data/raw/train/NoF/img_00008.jpg")
    pad_directory(balanced_in_list[d], padded_balanced_out_list[d], 50)
    print(d)

0
1
2
3
4
5
6
7


In [10]:
imhistmatch_direc('../data/raw/test_stg1/','../data/processed/test/histmatch/', '../data/raw/train/NoF/img_00008.jpg')
pad_directory('../data/processed/test/histmatch/', '../data/processed/test/histmatch_padded/', 50)